In [2]:
import nltk
import numpy as np
import pandas as pd
import re
import time
from lightgbm import LGBMClassifier
from nltk.corpus import wordnet, stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold
from xgboost import XGBClassifier

In [3]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Загрузим данные:

In [3]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
f"{df.toxic.mean():.1%} токсичных комментариев"

'10.2% токсичных комментариев'

Имеем датасет с английскими комментариями в которых ~ 10% токсичных комментов;  
При этом содержится много не нужных символов, треубется преобработоать текст.

### Очистка `text`

Проведем текст к нижнему регистру, очистим от линщних знаков:

In [7]:
def cleaner(row):
    text = row['text'].lower()
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-z ]+", "", text).strip()
    return text

In [8]:
df['cleaned_text'] = df.apply(cleaner, axis=1)

In [9]:
df.sample(4)

,text,toxic,cleaned_text
131826,How do you create a project? Because i'm tryin...,0,how do you create a project because im trying ...
61983,So does this mean I have been Wiki-vanquished?,0,so does this mean i have been wikivanquished
116101,"Welcome!\n\nHello, , and welcome to Wikipedia!...",0,welcome hello and welcome to wikipedia thank...
16477,"Yes, but this encyclopedia articles at their b...",0,yes but this encyclopedia articles at their be...


### Леммитизация

Определим функции `lemmatization` и `pos_tag_wordnet`:

In [10]:
def pos_tag_wordnet(text):
    """
        Create pos_tag with wordnet format
    """
    wordnet_map = {
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "J": wordnet.ADJ,
        "R": wordnet.ADV
    }

    pos_tagged_text = nltk.pos_tag(text)

    pos_tagged_text = [
        (word,
         wordnet_map.get(pos_tag[0])) if pos_tag[0] in wordnet_map.keys() else
        (word, wordnet.NOUN) for (word, pos_tag) in pos_tagged_text
    ]

    return pos_tagged_text


def lemmatization(clmn: pd.Series):

    wordnet_lemmatizer = WordNetLemmatizer()

    new_result = []

    for phrase in clmn:
        pos_tag = pos_tag_wordnet([i for i in phrase.split()])
        new_phrase = str()
        for word in pos_tag:
            new_phrase += " " + (wordnet_lemmatizer.lemmatize(word[0],
                                                              pos=word[1]))
        new_result.append(new_phrase)

    return pd.Series(new_result)

In [11]:
%%time
df['lemmas'] = lemmatization(df['cleaned_text'])

Wall time: 6min 11s


In [12]:
df_lemm = df.drop(columns=['text', 'cleaned_text'])

In [13]:
df_lemm.head()

,toxic,lemmas
0,0,explanation why the edits make under my usern...
1,0,daww he match this background colour im seemi...
2,0,hey man im really not try to edit war it just...
3,0,more i cant make any real suggestion on impro...
4,0,you sir be my hero any chance you remember wh...


## Обучение

### Признаки TF-IDF

In [14]:
train, test = train_test_split(df_lemm,
                               test_size=0.25,
                               random_state=42,
                               stratify=df_lemm['toxic'])

In [15]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Администратор\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
tf_idf_train = count_tf_idf.fit_transform(train['lemmas'])
tf_idf_test = count_tf_idf.transform(test['lemmas'])

In [17]:
print("Размер матрицы признаков обучающей выборки:", tf_idf_train.shape)
print("Размер матрицы признаков тестовой выборки:", tf_idf_test.shape)

Размер матрицы признаков обучающей выборки: (119678, 175251)
Размер матрицы признаков тестовой выборки: (39893, 175251)


### Logistic Regression

Обучим модель логистической регрессии:

In [18]:
log_reg = LogisticRegression(random_state=42,
                                    solver='liblinear',
                                    max_iter=500,
                                    verbose=True)

params = [{
    'penalty': ['l1'],
    'solver': ['liblinear'],
    'C': [0.001, 0.01, 0.1, 1, 10, 50, 100, 200]
}]

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

grid_log_reg = GridSearchCV(log_reg,
                            param_grid=params,
                            scoring='f1',
                            cv=cv,
                            verbose=True,
                            n_jobs=-1)

In [19]:
%%time
start = time.time()

grid_log_reg.fit(tf_idf_train, train.toxic)

end = time.time()
log_reg_fit_time = end - start

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[LibLinear]Wall time: 45.5 s


In [20]:
best_grid_log_reg = grid_log_reg.best_estimator_

### XGBClassifier

Обучим модель eXtreme Gradient Boosting Сlassifier

In [21]:
%%time
start = time.time()

xgbc = XGBClassifier()
params = {'n_estimators': [150, 200, 350],
          'learning_rate' : [0.1, 0.3, 0.5],
          'max_depth': [3],
          'class_weight' : ['balanced'],}
xgbc_model = GridSearchCV(estimator=xgbc, param_grid=params, n_jobs=-1, cv=5, scoring='f1')
xgbc_model.fit(tf_idf_train, train['toxic'])

end = time.time()
xgbc_fit_time = end - start

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:37:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "class_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:37:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 18min


### LGBMClassifier

Обучим модель Light Gradient Boosted Machine

In [22]:
%%time
start = time.time()

gbm = LGBMClassifier()
params = {'num_leaves': [50],
          'learning_rate' : [0.1, 0.3, 0.5],
          'n_estimators': [100],
         'metric': ['F1']}
gbm_model = GridSearchCV(estimator=gbm, param_grid=params, n_jobs=-1, cv=5)
gbm_model.fit(tf_idf_train, train['toxic'])

end = time.time()
gbm_fit_time = end - start

Wall time: 5min 18s


## Выводы

Построим таблицу с результатами:

In [23]:
# Logistic Regression
f1_log_reg = f1_score(test['toxic'], best_grid_log_reg.predict(tf_idf_test))
# XGBClassifier
f1_xgbc = f1_score(test['toxic'], xgbc_model.predict(tf_idf_test))
# LGBMClassifier
f1_gbm = f1_score(test['toxic'], gbm_model.predict(tf_idf_test))

In [24]:
model_results = {
                'F1 score': [f1_log_reg, f1_xgbc, f1_gbm],
                'Time, sec': [log_reg_fit_time, xgbc_fit_time, gbm_fit_time]
                 }

In [25]:
results = pd.DataFrame(data=model_results, index=('Logistic Regression', 'XGBClassifier', 'LGBMClassifier'))
results

,F1 score,"Time, sec"
Logistic Regression,0.775799,45.522501
XGBClassifier,0.761852,1080.018915
LGBMClassifier,0.768203,318.196656


Вывод:
Были обучены 3 модели для классификации комментариев на позитивные и негативные.  
Лучшая модель Logistic Regression с F1-score: 0.78, что соответвуеют поставленной задаче.  
При этом XGBClassifier и LGBMClassifier показали F1-score 0.76 и 0.77 соответственно, но большим временем обучения.